# Dynamic Programming

Dynamic programming is used when we need to access the same sub-problem more than once ("memorizing") because sub-problem overlap. If sub-problems do not overlap, we use divide and conquer.

In [1]:
from ipynb.fs.full.DataStructures import *

In [2]:
import numpy as np
import pandas as pd
import itertools

## 0-1 Knapsack
Brute force

In [3]:
list(itertools.combinations([1,2,3], 2))

[(1, 2), (1, 3), (2, 3)]

In [4]:
def zeroOneKnapsack(values, weights, capacity):
    best_value = 0
    best_combo = ()
    items = list(range(len(values)))
    for i in range(1,len(values)+1):
        # trying combinations with i objects
        for combo in itertools.combinations(items, i):
            vals = [values[i] for i in combo]
            wts = [weights[i] for i in combo]
            total_weight = sum(wts)
            total_vals = sum(vals)
            
            if total_vals > best_value and total_weight <= capacity:
                best_value = total_vals
                best_combo = combo
    
    return best_combo, best_value

In [5]:
values = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50

In [6]:
zeroOneKnapsack(values, weights, capacity)

((1, 2), 220)

## Fractional Knapsack

In [7]:
def fractionalKnapsack(values, weights, capacity):
    items = list(range(1, len(values)+1))
    knapsack = []
    value_per_weight = pd.Series([values[i]/weights[i] for i in range(len(values))])
    
    while len(knapsack) < capacity:
        to_take_idx = pd.Index(value_per_weight).get_loc(max(value_per_weight[[w > 0 for w in weights]]))
        weights[to_take_idx] -= 1
        knapsack.append(items[to_take_idx])
        
    return knapsack

In [8]:
values = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50

knapsack = fractionalKnapsack(values, weights, capacity)
knapsack = pd.Series(knapsack)

sum(knapsack == 1), sum(knapsack == 2), sum(knapsack == 3)

(10, 20, 20)

## Dynamic Knapsack

In [9]:
class Solution():
    def __init__(self, combo = [], weight = 0, value = 0):
        self.combo = combo
        self.weight = weight
        self.value = value
        
    def __repr__(self):
        if type(self.combo) == int:
            return str(self.combo)
        return '&'.join([str(i) for i in self.combo])


def dynamicKnapsack(values, weights, capacity):
    # sols[i, j] is the intermediate solution using the first i values and capacity j
    sols = [[Solution() for i in range(capacity + 1)] for j in range(len(values))]

    # fill out sols[0, j] consider first item only
    for c in range(len(sols[0])):
        if weights[0] <= c:
            sols[0][c].combo = [0]
            sols[0][c].weight = weights[0]
            sols[0][c].value = values[0]

    # fill out sols[i, 0] consider weight 0
    # no items should have weight 0 so can't take anything!
    for i in range(len(sols)):
        sols[i][0].combo = []
        sols[i][0].weight = 0
        sols[i][0].value = 0

    # fill out sols[i, j]
    for i in range(1, len(sols)):
        for j in range(1, len(sols[0])):
            if weights[i] > j:
                sols[i][j] = sols[i - 1][j]
            else:
                v1 = sols[i - 1][j].value
                v2 = sols[i - 1][j - weights[i]].value + values[i]
                if v1 > v2:
                    sols[i][j] = sols[i-1][j]
                else:
                    sols[i][j].combo = sols[i - 1][j - weights[i]].combo + [i]
                    sols[i][j].weight = sols[i - 1][j - weights[i]].weight + weights[i]
                    sols[i][j].value = sols[i - 1][j - weights[i]].value + values[i]
    
    return sols[-1][-1]

In [10]:
values = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50

In [11]:
best = dynamicKnapsack(values, weights, capacity)
best, best.value, best.weight

(1&2, 220, 50)

## Dijkstra's Algorithm

In [12]:
def dijkstra(graph, start):
    start = [v for v in vertices if v.v == start][0]

    start.dist = 0

    queue = [start]

    while len(queue) > 0:
        curr = queue.pop(0)
        curr.checked = True
        curr_edges = graph.es_from_v_directed(curr)
        for edge in curr_edges:
            if not edge.v2.checked and (
                edge.v2.dist == None or 
                edge.v1.dist + edge.weight < edge.v2.dist
            ):
                edge.v2.dist = edge.v1.dist + edge.weight

        queue.extend([c.v2 for c in 
                      sorted(curr_edges, key = lambda e: e.v2.dist) 
                      if not c.v2.checked
        ])

Directed Graph from side 240. Edge points from v1 to v2.

In [13]:
vertices = [Vertex(v) for v in ['A','B','C','D','E','F']]

edges = []
edges.append(Edge(vertices[0],vertices[1],4))
edges.append(Edge(vertices[0],vertices[2],2))
edges.append(Edge(vertices[1],vertices[2],5))
edges.append(Edge(vertices[1],vertices[3],10))
edges.append(Edge(vertices[2],vertices[4],3))
edges.append(Edge(vertices[3],vertices[5],11))
edges.append(Edge(vertices[4],vertices[3],4))

graph1 = Graph(vertices, edges)

In [14]:
dijkstra(graph1, start = "A")

In [15]:
graph1.vertices[-1].dist

20

Graph from slide 241 - 246 example

In [16]:
vertices = [Vertex(v) for v in ['s','t','y','x','z']]

edges = []
edges.append(Edge(vertices[0],vertices[1],10))
edges.append(Edge(vertices[0],vertices[2],5))
edges.append(Edge(vertices[1],vertices[2],2))
edges.append(Edge(vertices[2],vertices[1],3))
edges.append(Edge(vertices[1],vertices[3],1))
edges.append(Edge(vertices[2],vertices[4],2))
edges.append(Edge(vertices[2],vertices[3],9))
edges.append(Edge(vertices[3],vertices[4],4))
edges.append(Edge(vertices[4],vertices[3],6))
edges.append(Edge(vertices[4],vertices[0],7))

graph2 = Graph(vertices, edges)

In [17]:
dijkstra(graph2, start = "s")

In [18]:
graph2.vertices[-1].dist

7

## Sequence Alignment

In [19]:
def direct_align(seq1, seq2):
    return sum([seq1[i] == seq2[i] for i in range(min(len(seq1), len(seq2)))])        

In [20]:
direct_align("mathamatiks", "mathematics")

9

In [21]:
direct_align("mathmaticks", "mathematics")

5

In [22]:
def smith_waterman(seq1, seq2, sim_score, gap_penalty):
    # score[i, j] is between ith letter in seq1 and jth in seq 2
    score = [[-1 for j in range(len(seq2) + 1)] for i in range(len(seq1) + 1)]
    score[0] = [0 for i in range(len(seq2) + 1)]
    for i in range(len(seq1) + 1):
        score[i][0] = 0

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            opt1 = score[i-1][j-1] + sim_score(seq1[i-1], seq2[j-1])
            opt2s = [score[i-k][j] - gap_penalty(k) for k in range(1, i+1)]
            opt2 = max(opt2s)
            opt3s = [score[i][j-k] - gap_penalty(k) for k in range(1, j+1)]
            opt3 = max(opt3s)
            opt4 = 0

            score[i][j] = max([opt1, opt2, opt3, opt4])
    

In [23]:
def smith_waterman(seq1, seq2, sim_score, gap_penalty):
    score = build_score(seq1, seq2, sim_score, gap_penalty)
    return traceback(score, seq1, seq2)

def build_score(seq1, seq2, sim_score, gap_penalty):
    # score[i, j] is between ith letter in seq1 and jth in seq 2
    score = [[-1 for j in range(len(seq2) + 1)] for i in range(len(seq1) + 1)]
    score[0] = [0 for i in range(len(seq2) + 1)]
    for i in range(len(seq1) + 1):
        score[i][0] = 0

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            opt1 = score[i-1][j-1] + sim_score(seq1[i-1], seq2[j-1])
            opt2s = [score[i-k][j] - gap_penalty(k) for k in range(1, i+1)]
            opt2 = max(opt2s)
            opt3s = [score[i][j-k] - gap_penalty(k) for k in range(1, j+1)]
            opt3 = max(opt3s)
            opt4 = 0

            score[i][j] = max([opt1, opt2, opt3, opt4])
            
    return score
        
def get_max_idx(matrix):
    matrix = pd.DataFrame(matrix)
    m = max(matrix.max())
    rows = pd.Series([m in matrix.iloc[i].values for i in range(matrix.shape[0])])
    row = pd.Index(rows).get_loc(True)
    col = pd.Index(matrix.iloc[row]).get_loc(m)
    return row, col

def traceback(score, seq1, seq2):
    out1 = ''
    out2 = ''

    i, j = get_max_idx(score)

    while True:
        up = score[i][j-1]
        diag = score[i-1][j-1]
        left = score[i-1][j]

        if up <= 0 and diag <= 0 and left <= 0:
            i -= 1
            j -= 1
            out1 = seq1[i] + out1
            out2 = seq2[j] + out2
            break

        m = max(up, diag, left)
        if m == up:
            j -= 1
            out1 = '-' + out1
            out2 = seq2[j] + out2
        elif m == diag:
            i -= 1
            j -= 1
            out1 = seq1[i] + out1
            out2 = seq2[j] + out2
        elif m == left:
            i -= 1
            out1 = seq1[i] + out1
            out2 = '-' + out2
            
    return out1, out2

example from slide 252

In [24]:
seq2 = "tgttacgg"
seq1 = "ggttgacta"

def sim_score(a, b):
    if a == b:
        return 3
    else:
        return -3

def gap_penalty(k):
    return 2*k

In [25]:
smith_waterman(seq1, seq2, sim_score, gap_penalty)

('ggttgac', 'g-tt-ac')

example from slide 249

In [26]:
smith_waterman('mathmaticks', 'mathematics', sim_score, gap_penalty)

('math-maticks', 'mathematic-s')